# Model Creation and Training

In [3]:
import pandas as pd
import numpy as np
import logging
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

logging.basicConfig(filename = "logs.log", format = "%(asctime)s -- %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.INFO)

In [7]:
training_data = pd.read_csv("training_data/Attempt1.csv")
training_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-03-12,2559.660645,2606.438721,2559.126953,2574.754150,2574.754150,6532996574
1,2022-03-13,2573.488037,2594.549805,2503.885254,2518.944580,2518.944580,8632000379
2,2022-03-14,2518.486328,2604.034424,2505.299316,2590.696045,2590.696045,11244398839
3,2022-03-15,2590.668945,2662.329590,2515.765869,2620.149658,2620.149658,12861105614
4,2022-03-16,2620.028564,2781.307129,2610.764404,2772.055664,2772.055664,17915109769


In [11]:
### Load_Data ###
training_data = pd.read_csv("training_data/Attempt1.csv")
X = training_data.loc[:, ["Date", "Open"]]
y = training_data.loc[:, "Close"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)
logging.info("Dataset Upload successfully")
logging.info(f"X training set shape:{X_train.shape}")
logging.info(f"y training set shape:{y_train.shape}")
logging.info(f"X test set shape:{X_test.shape}")
logging.info(f"y test set shape:{y_test.shape}")


In [ ]:
### Variables ###
batch_size = 256
epochs = 100
attention_heads = 4
projection_dim = 128
dropout = 0.1
conv_dim = 4
num_transformer_blocks = 8
mlp_units = [2048, 1024]
tranformer_mlp_units = [projection_dim ** 2, projection_dim]


In [22]:
def positional_encoding(inputs):
    input_projection = layers.Dense(projection_dim)
    embedding = layers.Embedding(input_dim = 1, output_dim = projection_dim)
    encoded_batch = []

In [ ]:
def transformer_encoder(inputs, attention_heads, projection_dim, conv_dim, dropout):
    ### Layer Normalization / Multihead Attention Layers ###
    x = layers.LayerNormalization(epsilon = 1e-6)(inputs)
    x = layers.MultiHeadAttention(num_heads = attention_heads, key_dim = projection_dim, dropout = dropout)(x,x)
    skip1 = layers.Add()([x, inputs])
    
    ### Feed Forward ###
    x = layers.LayerNormalization(epsilon = 1e-6)(skip1)
    x = layers.Conv1D(filters=conv_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    skip2 = layers.Add()([x,skip1])
    
    return skip2

In [ ]:
def build_model():
    pass

In [ ]:
def train_model(model):
    optimizer = tf.optimizers.Adam(learning_rate=1e-3, decay = 1e-4)
    checkpoint_path = "/models/"
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = "val_Accuracy", save_best_only = True, save_weights_only = True)
    model.compile(optimizer=optimizer, 
                  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics = [tf.keras.metrics.CategoricalAccuracy(name = "Accuracy")])
    
    history = model.fit(
        x = X_train,
        y = y_train,
        batch_size = batch_size,
        epochs = epochs,
        validation_split = 0.2,
        callbacks = [checkpoint_callback],
        
    )
    
    return history